In [618]:
import numpy as np
import pandas as pd
import random

In [619]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('hw2_question1.csv', header=None, names=None)
dataset = np.array(df).tolist()

for i in range(len(dataset)):
    for j in range(len(dataset[0])):
        dataset[i][j] = str(dataset[i][j])        
        
training_data = []; test_data = []
m = []; b = []
for row in dataset:
    if row[-1] == 2:
        b.append(row)
    else:
        m.append(row)
for row in m:
    if random.random() < 0.66:
        training_data.append(row)
    else:
        test_data.append(row)
for row in b:
    if random.random() < split:
        training_data.append(row)
    else:
        test_data.append(row)                

In [620]:
header = ["Clump Thickness", "Uniformity of Cell Size", "Uniformity of Cell Shape",
          "Marginal Adhesion", "Single Epithelial Cell Size",
          "Bare Nuclei", "Bland Chromatin", "Normal Nucleoli",
          "Mitoses", "label"]

In [621]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [622]:
def class_counts(rows):
    counts = {}
    if isinstance(rows[0], list) == False:
        counts[rows[-1]] = 0
        counts[rows[-1]] += 1
    else:
        for row in rows:
            label = row[-1]
            if label not in counts:
                counts[label] = 0
            counts[label] += 1
    return counts

In [623]:
class SplitCriteria:

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        return val == self.value

    def __repr__(self):
        condition = "="
        return "%s %s %s" % (
            header[self.column], condition, str(self.value))

In [624]:
def partition(rows, splitCriteria):
    true_rows, false_rows = [], []
    for row in rows:
        if splitCriteria.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [625]:
import math
def entropy(left, right):
    left2=0;left4=0;right2=0;right4=0
    for row in left:
        if row[-1] == '4':
            left4 += 1
        else:
            left2 += 1

        left_hx = (left2 /len(left) ) * np.log2(left2 / len(left)) \
               + (left4 / len(left)) * np.log2(left4 / len(left))
        left_hx = -1*left_hx

    for row in right:
        if row[-1] == '4':
            right4 += 1
        else:
            right2 += 1

        right_hx = (right2 /len(right) ) * np.log2(right2 / len(right)) \
               + (right4 / len(right)) * np.log2(right4 / len(right))
        right_hx = -1*right_hx

    total = len(left) + len(right)
    if math.isnan(left_hx):
        left_hx=0
    if math.isnan(right_hx):
        right_hx=0
    return float(left_hx*(len(left)/total) + right_hx*(len(right)/total))

In [626]:
def gini(left,right):
    left2=0;left4=0;right2=0;right4=0
    for row in left:
        if row[-1] == '4':
            left4 += 1
        else:
            left2 += 1
    left_gini = (left2 /len(left)) * (1-(left2 /len(left))) \
                + (left4 /len(left)) * (1-(left4 /len(left)))
        
    for row in right:
        if row[-1] == '4':
            right4 += 1
        else:
            right2 += 1
    right_gini = (right2 /len(right)) * (1-(right2 /len(right))) \
                + (right4 /len(right)) * (1-(right4 /len(right)))
        
    total = len(left) + len(right)
    if math.isnan(left_gini):
        left_hx=0
    if math.isnan(right_gini):
        right_hx=0
    return float(left_gini*(len(left)/total) + right_gini*(len(right)/total))

In [627]:
def find_split(rows):
    best_gain = 10 
    best_splitCriteria = None  
    n_features = len(rows[0]) - 1  

    for col in range(n_features):  
        values = set([row[col] for row in rows])
        for val in values:  
            splitCriteria = SplitCriteria(col, val)
            true_rows, false_rows = partition(rows, splitCriteria)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = gini(true_rows, false_rows)
            if gain < best_gain:
                best_gain, best_splitCriteria = gain, splitCriteria

    return best_gain, best_splitCriteria

In [628]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [629]:
class Decision_Node:
    def __init__(self,
                 splitCriteria,
                 true_branch,
                 false_branch):
        self.splitCriteria = splitCriteria
        self.true_branch = true_branch
        self.false_branch = false_branch

In [630]:
def build_tree(rows):

    gain, splitCriteria = find_split(rows)
    if gain == 10:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, splitCriteria)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(splitCriteria, true_branch, false_branch)

In [631]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    print(spacing + str(node.splitCriteria))

    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [632]:
my_tree = build_tree(training_data)

In [633]:
print_tree(my_tree)

Bare Nuclei = 1
--> True:
  Uniformity of Cell Size = 10
  --> True:
    Clump Thickness = 10
    --> True:
      Uniformity of Cell Shape = 7
      --> True:
        Predict {'4': 1}
      --> False:
        Marginal Adhesion = 1
        --> True:
          Predict {'4': 1}
        --> False:
          Marginal Adhesion = 8
          --> True:
            Predict {'4': 1}
          --> False:
            Predict {'4': 1}
    --> False:
      Predict {'4': 1}
  --> False:
    Uniformity of Cell Size = 6
    --> True:
      Clump Thickness = 5
      --> True:
        Predict {'4': 1}
      --> False:
        Predict {'4': 1}
    --> False:
      Single Epithelial Cell Size = 6
      --> True:
        Clump Thickness = 7
        --> True:
          Predict {'4': 1}
        --> False:
          Predict {'4': 1}
      --> False:
        Marginal Adhesion = 10
        --> True:
          Predict {'4': 1}
        --> False:
          Marginal Adhesion = 8
          --> True:
            Pred

In [634]:
def classify(row, node):

    if isinstance(node, Leaf):
        return node.predictions

    if node.splitCriteria.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [635]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [636]:
for row in test_data:
    print("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: 2. Predicted: {'4': '100%'}
Actual: 2. Predicted: {'2': '100%'}
Actual: 2. Predicted: {'2': '100%'}
Actual: 2. Predicted: {'2': '100%'}
Actual: 4. Predicted: {'2': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 2. Predicted: {'2': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 2. Predicted: {'2': '100%'}
Actual: 2. Predicted: {'2': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'2': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'4': '100%'}
Actual: 4. Predicted: {'2': '100%'}
Actual: 2. Predicted: {'4': '100%'}
Actual: 2. Predicted: {'2': '100%'}
Actual: 2. Predicted: {'2': '100%'}
Actual: 2. Predicted: {'2': 